# DAT210x - Programming with Python for DS

## Module6- Lab4

This code is intentionally missing! Read the directions on the course lab page!

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np 
import time
C = 1
kernel = 'linear'
# TODO: Change to 200000 once you get to Question#2
iterations = 5000

# You can set this to false if you want to draw the full square matrix:
FAST_DRAW = True
def drawPlots(model, X_train, X_test, y_train, y_test, wintitle='Figure 1'):
    # You can use this to break any higher-dimensional space down,
    # And view cross sections of it.

    # If this line throws an error, use plt.style.use('ggplot') instead
    mpl.style.use('ggplot') # Look Pretty

    padding = 3
    resolution = 0.5
    max_2d_score = 0

    y_colors = ['#ff0000', '#00ff00', '#0000ff']
    my_cmap  = mpl.colors.ListedColormap(['#ffaaaa', '#aaffaa', '#aaaaff'])
    colors   = [y_colors[i] for i in y_train]
    num_columns = len(X_train.columns)

    fig = plt.figure()
    fig.canvas.set_window_title(wintitle)
    fig.set_tight_layout(True)
    
    cnt = 0
    for col in range(num_columns):
        for row in range(num_columns):
            
            # Easy out
            if FAST_DRAW and col > row:
                cnt += 1
                continue

            ax = plt.subplot(num_columns, num_columns, cnt + 1)
            plt.xticks(())
            plt.yticks(())

            # Intersection:
            if col == row:
                plt.text(0.5, 0.5, X_train.columns[row], verticalalignment='center', horizontalalignment='center', fontsize=12)
                cnt += 1
                continue


            # Only select two features to display, then train the model
            X_train_bag = X_train.ix[:, [row,col]]
            X_test_bag = X_test.ix[:, [row,col]]
            model.fit(X_train_bag, y_train)

            # Create a mesh to plot in
            x_min, x_max = X_train_bag.ix[:, 0].min() - padding, X_train_bag.ix[:, 0].max() + padding
            y_min, y_max = X_train_bag.ix[:, 1].min() - padding, X_train_bag.ix[:, 1].max() + padding
            xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution),
                                 np.arange(y_min, y_max, resolution))

            # Plot Boundaries
            plt.xlim(xx.min(), xx.max())
            plt.ylim(yy.min(), yy.max())

            # Prepare the contour
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            plt.contourf(xx, yy, Z, cmap=my_cmap, alpha=0.8)
            plt.scatter(X_train_bag.ix[:, 0], X_train_bag.ix[:, 1], c=colors, alpha=0.5)


            score = round(model.score(X_test_bag, y_test) * 100, 3)
            plt.text(0.5, 0, "Score: {0}".format(score), transform = ax.transAxes, horizontalalignment='center', fontsize=8)
            max_2d_score = score if score > max_2d_score else max_2d_score

            cnt += 1

    print("Max 2D Score: ", max_2d_score)
    
def benchmark(model, X_train, X_test, y_train, y_test, wintitle='Figure 1'):
    print(wintitle + ' Results')
    s = time.time()
    
    for i in range(iterations):
        # TODO: train the classifier on the training data / labels:
        model.fit(X_train,y_train)
        # .. your code here ..
        
    print("{0} Iterations Training Time: ".format(iterations), time.time() - s)


    s = time.time()
    for i in range(iterations):
        # TODO: score the classifier on the testing data / labels:
        score = model.score(X_test, y_test)
        # .. your code here ..
        
    print("{0} Iterations Scoring Time: ".format(iterations), time.time() - s)
    print("High-Dimensionality Score: ", round((score*100), 3))
    
X = pd.read_csv('Datasets/wheat.data', index_col='id')
# An easy way to show which rows have nans in them:
X[pd.isnull(X).any(axis=1)]
X.dropna(axis=0, inplace=True)
X[pd.isnull(X).any(axis=1)]
X.wheat_type = X.wheat_type.map({'canadian':0, 'kama':1, 'rosa':2})
y = X.wheat_type.copy()
X.drop('wheat_type', axis=1, inplace=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=7)

from sklearn import tree
dt = tree.DecisionTreeClassifier(max_depth=1, random_state=2)
#dt.fit(X,y)
tree.export_graphviz(dt.tree_, out_file='tree.dot', feature_names=X.columns)
#from subprocess import call
#call(['dot', '-T', 'png', 'C:\Users\ferryto\Documents\GitHub\DAT210x\Module6\tree.dot', '-o', 'C:\Users\ferryto\Documents\GitHub\DAT210x\Module6\tree.png'])
benchmark(dt, X_train, X_test, y_train, y_test, 'Tree')
#drawPlots(dt, X_train, X_test, y_train, y_test, 'Tree')

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
benchmark(knn, X_train, X_test, y_train, y_test, 'KNeighbors')
#drawPlots(knn, X_train, X_test, y_train, y_test, 'KNeighbors')

from sklearn.svm import SVC
svc = SVC(C=1, kernel='linear')
benchmark(svc, X_train, X_test, y_train, y_test, 'SVC')
#drawPlots(svc, X_train, X_test, y_train, y_test, 'SVC')
#plt.show()

Tree Results
('5000 Iterations Training Time: ', 1.4219999313354492)
('5000 Iterations Scoring Time: ', 0.8640000820159912)
('High-Dimensionality Score: ', 68.852)
KNeighbors Results
('5000 Iterations Training Time: ', 1.4920001029968262)
('5000 Iterations Scoring Time: ', 5.565999984741211)
('High-Dimensionality Score: ', 83.607)
SVC Results
('5000 Iterations Training Time: ', 2.873000144958496)
('5000 Iterations Scoring Time: ', 1.1319999694824219)
('High-Dimensionality Score: ', 86.885)
